In [ ]:
# tips on using Keras models with TensorFlow backend in TF Serving taken from here:
# https://medium.com/@mr.acle/exporting-deep-learning-models-from-keras-to-tensorflow-serving-7d4a6e49ce3

import tensorflow as tf

sess = tf.Session()

tf.keras.backend.set_session(sess)
tf.keras.backend._LEARNING_PHASE = tf.constant(0)
tf.keras.backend.set_learning_phase(0)

# see below: `load_model` might not be the best way really
model = tf.keras.models.load_model("../models/model-improvement-05-0.99.hdf5")

input_dict = {'image': model.input[0], 'meta': model.input[1]}
output_dict = {"prediction": model.output}
prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def(input_dict, output_dict)

export_path = "models/cheetah_inception/1"
builder = tf.saved_model.builder.SavedModelBuilder(export_path)

legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

# this step is strange: we need to initialize something, else TF complains
# (even though a complete compiled model was loaded with `load_model(..)`)
# but the initializiation leads to random results at prediction
# therefore, we reload the weights manually afterwards
# Maybe better in the future: Load architecture and weights explicitly, don't use `load_model`
sess.run(init_op)
model.load_weights("../models/weights-improvement-05-0.99.hdf5")

# Add the meta_graph and the variables to the builder
builder.add_meta_graph_and_variables(
      sess, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
           tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
               prediction_signature,
      },
      legacy_init_op=legacy_init_op)
# save the graph      
builder.save() 